In [13]:
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

#preprocessing
def tokenize(sentence):
    return nltk.word_tokenize(sentence)


def stem(word):
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [14]:
#nltk.download('punkt')

In [15]:
import numpy as np
import random
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

#from nltk_utils import bag_of_words, tokenize, stem
#from model import NeuralNet

with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag)) #mapping b/w i/p & o/p in training

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)  #length of the sample
        self.x_data = X_train          #Feature patterns stored in x_data
        self.y_data = y_train          # The output tagsstored in y_data

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    from sklearn import tree
    model = tree.DecisionTreeClassifier()
    model.fit(X_train,y_train)

    

94 patterns
16 tags: ['Cold_drinks', 'French fries', 'chicken_momo', 'corn_momo', 'delivery', 'goodbye', 'greeting', 'items', 'menu_list', 'paneer_momo', 'payments', 'price', 'thanks', 'timing', 'veg_momo', 'wb']
100 unique stemmed words: ["'s", 'a', 'accept', 'allow', 'anyon', 'are', 'avail', 'buy', 'bye', 'can', 'card', 'cash', 'chicken', 'close', 'cod', 'cold', 'corn', 'cost', 'credit', 'day', 'deliveri', 'do', 'doe', 'drink', 'eat', 'food', 'french', 'fri', 'get', 'give', 'good', 'goodby', 'great', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'in', 'is', 'it', 'item', 'kind', 'later', 'list', 'long', 'look', 'lot', 'mastercard', 'me', 'menu', 'momo', 'morn', 'much', 'my', 'na', 'need', 'now', 'of', 'ok', 'okay', 'on', 'onli', 'open', 'order', 'paneer', 'pay', 'payment', 'paypal', 'price', 'procedur', 'remain', 'restur', 'see', 'sell', 'serv', 'ship', 'show', 'take', 'thank', 'that', 'the', 'there', 'time', 'to', 'varieti', 'veg', 'wan', 'want', 'what', 'when', 'which', 'will',

In [22]:
#from sklearn import tree
#model = tree.DecisionTreeClassifier()
#from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
#model = RandomForestClassifier(n_estimators=20)
#model = svm.SVC(kernel='linear').fit(X_train,y_train)
#model = LogisticRegression()
model = KNeighborsClassifier()
model.fit(X_train,y_train)

KNeighborsClassifier()

In [23]:
bot_name = "Oreo"
print("Let's chat! (type 'done' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "done":
        print("Thank you! Visit Again.")
        break

    sentence = tokenize(sentence)
    print(sentence)
    X = bag_of_words(sentence, all_words)
    print(X)
    X = X.reshape(1, X.shape[0])
    print(X)
    xx = model.predict(X)
    print(tags[xx[0]])
    #X = torch.from_numpy(X).to(device)
    #print(X)
    
    
    
    #probs = torch.softmax(output, dim=1)#http://localhost:8888/notebooks/TT%20Chatbot%20Oreo.ipynb#
    #prob = probs[0][predicted.item()]
    #if prob.item() > 0.75:
    for intent in intents['intents']:
        if tags[xx[0]] == intent["tag"]:
           # print(prob.item())
            print(f"{bot_name}: {random.choice(intent['responses'])}")
    #else:
        #print(prob.item())
        #print(f"{bot_name}: I do not understand...")


Let's chat! (type 'done' to exit)
You: Hi
['Hi']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]
goodbye
Oreo: Hope to meet you soon!!
You: Hellow
['Hellow']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [6]:
print(sentence)

done


In [11]:
x = model.predict([[0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])

In [12]:
print(x[0])

12


In [ ]:
print(tags[x[0]])

In [ ]:
print(label)

In [ ]:
#Chat application
import random
import json

import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

from sklearn import tree
model =  tree.DecisionTreeClassifier()
#model.load_state_dict(model_state)
#model.eval()
model.fit(X_train,y_train)

bot_name = "Oreo"
print("Let's chat! (type 'done' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "done":
        print("Thank you! Visit Again.")
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model.predict(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)#http://localhost:8888/notebooks/TT%20Chatbot%20Oreo.ipynb#
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")